In [1]:
from datetime import datetime
import datetime as dt
import time
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd
import duckdb as db
import IPython as ip
import  os as os

In [2]:
sg = Subgrounds()
apiKey = input('API Key, please')

In [18]:
#for when we save the files
file = input('Selet a folder to save output') ##enter your file path here - the file is in the repo "summary_stats.csv".

*******************************************************************
*******************************************************************
LET'S LOOK AT SILO FIRST

In [3]:
#SILO subgraph: https://gateway.thegraph.com/api/[api-key]/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB
silo = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB')

##this captures the list of people who borrowed XAI against gOHM
borrows = silo.Query.borrows(
    orderBy='timestamp',
    orderDirection='desc',
    first=1000,
    #skip = skipValue,
    where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' }

)

borrowsDb = sg.query_df([
    borrows.id
    ,borrows.hash
    ,borrows.logIndex
    ,borrows.blockNumber
    ,borrows.timestamp
    ,borrows.account
    ,borrows.market
    ,borrows.market.rates
    ,borrows.asset
    ,borrows.amount
    ,borrows.amountUSD 
])
borrowsDb.head(100)

#this, in turn, pulls repayment events
repays = silo.Query.repays(
    orderBy='timestamp',
    orderDirection='desc',
    first=1000,
    #skip = skipValue,
    where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' }

)

repaysDb = sg.query_df([
    repays.id
    ,repays.hash
    ,repays.logIndex
    ,repays.blockNumber
    ,repays.timestamp
    ,repays.account
    ,repays.market
    ,repays.market.rates
    ,repays.asset
    ,repays.amount
    ,repays.amountUSD
    ,repays.position
])
repaysDb.head(100)

In [8]:
##this captures the ENTIRE list of people who borrowed XAI against gOHM
borrowTicker = 0
borrowslist = pd.DataFrame()
borrowsDb = pd.DataFrame()
borrowListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (borrowTicker)*(1000)
    borrows = silo.Query.borrows(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    borrowslist = sg.query_df([
        borrows.id
        ,borrows.hash
        ,borrows.logIndex
        ,borrows.blockNumber
        ,borrows.timestamp
        ,borrows.account
        ,borrows.market
        ,borrows.market.rates
        ,borrows.asset
        ,borrows.amount
        ,borrows.amountUSD
        ,borrows.position
    ])

    #borrowslist.sort_values(by=['borrows_timestamp'])

    borrowsDb=pd.concat([borrowsDb, borrowslist])
    recordTimestamp1 = borrowsDb.iat[borrowTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    borrowsDbLength = len(borrowsDb)
    borrowListLength = len(borrowslist)
    recordID = borrowsDb.iat[borrowTicker,0]
    borrow = borrowsDb.iat[borrowTicker,1]
    print("iterations: ", borrowTicker, "Lines skipped: ",skipValue, "records collected: ", borrowListLength, " - latest recordID: ",borrow, " ", recordID , " - borrows DB length: ", borrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", borrowTicker, "records collected: ", borrowListLength, " - userdb length: ", borrowsDbLength)
    borrowTicker = borrowTicker+1

    if daysAgo>daysLimit: exit =True
    if borrowListLength<1000: exit =True

borrowsDb.columns = borrowsDb.columns.str.replace("borrows", "transaction")
borrowsDb['event']='borrow'
print("Done. Total Records collected: ", borrowsDbLength)

iterations:  0 Lines skipped:  0 records collected:  96  - latest recordID:  0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e0445d6b6e8045f8daf92657   0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e0445d6b6e8045f8daf92657-105  - borrows DB length:  96  - latest record from:  2023-02-08 22:23:59 1 days ago
Done. Total Records collected:  96


In [9]:
borrowsDb.head(20)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,NaN,BORROWER,True,186000000000000000000000,0,0,1,0,0,borrow
1,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,NaN,BORROWER,True,186000000000000000000000,0,0,1,0,0,borrow
2,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,NaN,BORROWER,True,186000000000000000000000,0,0,1,0,0,borrow
3,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,NaN,BORROWER,True,186000000000000000000000,0,0,1,0,0,borrow
4,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,NaN,BORROWER,True,186000000000000000000000,0,0,1,0,0,borrow
5,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,NaN,BORROWER,True,186000000000000000000000,0,0,1,0,0,borrow
6,0x2c0d9ab92c0eef11fa3394b4a89a62c7ee27bd5b64a1...,0x2c0d9ab92c0eef11fa3394b4a89a62c7ee27bd5b64a1...,251,16588637,1675914791,0x804157889ac56a4e50278610c67960c1605a3264,2,2,0,1,...,NaN,BORROWER,True,1000000000000000000000,0,0,1,0,0,borrow
7,0x2c0d9ab92c0eef11fa3394b4a89a62c7ee27bd5b64a1...,0x2c0d9ab92c0eef11fa3394b4a89a62c7ee27bd5b64a1...,251,16588637,1675914791,0x804157889ac56a4e50278610c67960c1605a3264,2,2,0,1,...,NaN,BORROWER,True,1000000000000000000000,0,0,1,0,0,borrow
8,0x2c0d9ab92c0eef11fa3394b4a89a62c7ee27bd5b64a1...,0x2c0d9ab92c0eef11fa3394b4a89a62c7ee27bd5b64a1...,251,16588637,1675914791,0x804157889ac56a4e50278610c67960c1605a3264,2,2,0,1,...,NaN,BORROWER,True,1000000000000000000000,0,0,1,0,0,borrow
9,0x2c0d9ab92c0eef11fa3394b4a89a62c7ee27bd5b64a1...,0x2c0d9ab92c0eef11fa3394b4a89a62c7ee27bd5b64a1...,251,16588637,1675914791,0x804157889ac56a4e50278610c67960c1605a3264,2,2,0,1,...,NaN,BORROWER,True,1000000000000000000000,0,0,1,0,0,borrow


In [10]:
##this tracks the ENTIRE list of events for repaying the XAI repayed against gOHM
repayTicker = 0
repayslist = pd.DataFrame()
repaysDb = pd.DataFrame()
repayListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (repayTicker)*(1000)
    repays = silo.Query.repays(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    repayslist = sg.query_df([
        repays.id
        ,repays.hash
        ,repays.logIndex
        ,repays.blockNumber
        ,repays.timestamp
        ,repays.account
        ,repays.market
        ,repays.market.rates
        ,repays.asset
        ,repays.amount
        ,repays.amountUSD
        ,repays.position
    ])

    #repayslist.sort_values(by=['repays_timestamp'])

    repaysDb=pd.concat([repaysDb, repayslist])
    recordTimestamp1 = repaysDb.iat[repayTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    repaysDbLength = len(repaysDb)
    repayListLength = len(repayslist)
    recordID = repaysDb.iat[repayTicker,0]
    repay = repaysDb.iat[repayTicker,1]
    print("iterations: ", repayTicker, "Lines skipped: ",skipValue, "records collected: ", repayListLength, " - latest recordID: ",repay, " ", recordID , " - repays DB length: ", repaysDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", repayTicker, "records collected: ", repayListLength, " - userdb length: ", repaysDbLength)
    repayTicker = repayTicker+1

    if daysAgo>daysLimit: exit =True
    if repayListLength<1000: exit =True

repaysDb.columns = repaysDb.columns.str.replace("repays", "transaction")
repaysDb['event']='repay'
print("Done. Total Records collected: ", repaysDbLength)

iterations:  0 Lines skipped:  0 records collected:  12  - latest recordID:  0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad01a68715d0385e8b26cd   0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad01a68715d0385e8b26cd-242  - repays DB length:  12  - latest record from:  2022-11-25 13:03:47 77 days ago
Done. Total Records collected:  12


In [11]:
repaysDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
1,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
2,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
3,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
4,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
5,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
6,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,1669248167,BORROWER,True,0,0,0,1,1,0,repay
7,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,1669248167,BORROWER,True,0,0,0,1,1,0,repay
8,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,1669248167,BORROWER,True,0,0,0,1,1,0,repay
9,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,1669248167,BORROWER,True,0,0,0,1,1,0,repay


In [12]:
#this tracks the ENTIRE list of events for depositing gOHM for lending
depositTicker = 0
depositslist = pd.DataFrame()
depositsDb = pd.DataFrame()
depositListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (depositTicker)*(1000)
    deposits = silo.Query.deposits(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    depositslist = sg.query_df([
        deposits.id
        ,deposits.hash
        ,deposits.logIndex
        ,deposits.blockNumber
        ,deposits.timestamp
        ,deposits.account
        ,deposits.market
        ,deposits.market.rates
        ,deposits.asset
        ,deposits.amount
        ,deposits.amountUSD
        ,deposits.position
    ])

    #depositslist.sort_values(by=['deposits_timestamp'])

    depositsDb=pd.concat([depositsDb, depositslist])
    recordTimestamp1 = depositsDb.iat[depositTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    depositsDbLength = len(depositsDb)
    depositListLength = len(depositslist)
    recordID = depositsDb.iat[depositTicker,0]
    deposit = depositsDb.iat[depositTicker,1]
    print("iterations: ", depositTicker, "Lines skipped: ",skipValue, "records collected: ", depositListLength, " - latest recordID: ",deposit, " ", recordID , " - deposits DB length: ", depositsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", depositTicker, "records collected: ", depositListLength, " - userdb length: ", depositsDbLength)
    depositTicker = depositTicker+1
    if daysAgo>daysLimit: exit =True
    if depositListLength<1000: exit =True

#depositsDb.sort_values(by=['deposits_timestamp'])

depositsDb.columns = depositsDb.columns.str.replace("deposits", "transaction")
depositsDb['event']='deposit'
print("Done. Total Records collected: ", depositsDbLength)




iterations:  0 Lines skipped:  0 records collected:  222  - latest recordID:  0x7c0686952bdf22de3c8ce87e732690a4365c23f25939b0c34ffb9a2735dace81   0x7c0686952bdf22de3c8ce87e732690a4365c23f25939b0c34ffb9a2735dace81-204  - deposits DB length:  222  - latest record from:  2023-02-09 09:48:11 1 days ago
Done. Total Records collected:  222


In [13]:
depositsDb.head(20)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
1,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
2,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
3,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
4,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
5,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
6,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit
7,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit
8,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit
9,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit


In [14]:
siloEventsDb = pd.DataFrame()
siloEventsDb=pd.concat([depositsDb, borrowsDb,repaysDb])
siloEventsDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
1,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
2,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
3,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
4,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
5,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
6,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit
7,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit
8,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit
9,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,0x669ce641c8adcf855d5ea4fedda6b52fc63795fd0dd6...,369,16590634,1675938899,0x51bc3dd10a9ee4946f961038cf5d50057e49eb77,1,1,0,1,...,NaN,LENDER,True,9990000000000000000,1,0,0,0,0,deposit


In [15]:
#I swear I will figure out synthetic fields, but until then...
siloEventsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(transaction_timestamp::int) dateTime "
               "From siloEventsDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()
siloEventsDbTimed.head(100)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event,dateTime
0,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit,2023-02-09 17:48:11
1,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit,2023-02-09 17:48:11
2,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit,2023-02-09 17:48:11
3,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit,2023-02-09 17:48:11
4,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit,2023-02-09 17:48:11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0x7811e7757339551966af37751af4b56c16a3e141d682...,0x7811e7757339551966af37751af4b56c16a3e141d682...,149,16366137,1673229911,0x25fdda95197f804316f5ba8dbb66ac173ee19981,1,0,1,1,...,LENDER,True,0,1,1,0,0,0,deposit,2023-01-09 02:05:11
96,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,194,16235576,1671655835,0xdedcf5806c4968c6397eee97e68047bda339d0c1,14,12,2,19,...,LENDER,True,20000000000000000,1,0,0,0,0,deposit,2022-12-21 20:50:35
97,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,194,16235576,1671655835,0xdedcf5806c4968c6397eee97e68047bda339d0c1,14,12,2,19,...,LENDER,True,20000000000000000,1,0,0,0,0,deposit,2022-12-21 20:50:35
98,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,0xc094064b1815d07ca696c614f45dffbb054d2e5d5954...,194,16235576,1671655835,0xdedcf5806c4968c6397eee97e68047bda339d0c1,14,12,2,19,...,LENDER,True,20000000000000000,1,0,0,0,0,deposit,2022-12-21 20:50:35


In [16]:
siloEventsDblength = len(siloEventsDbTimed)
siloEventsDblength

330

In [19]:
path =file+'/siloFinance_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records.csv'
siloEventsDbTimed.to_csv(path, index = False)

path =file+'/siloFinance_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records_piped.csv'
siloEventsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

In [20]:
#let's pull interest rate data
interestRateTicker = 0
interestRateslist = pd.DataFrame()
interestRatesDb = pd.DataFrame()
interestRateListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (interestRateTicker)*(1000)
    interestRates = silo.Query.interestRates(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    interestRateslist = sg.query_df([
        interestRates.id
        ,interestRates.rate #5.21% should be stored as 5.21
        ,interestRates.duration #in days
        ,interestRates.maturityBlock #Maturity of the loan in block height. Only applies to fixed term lending (e.g. Notional)
        ,interestRates.side #The party the interest is paid to / received from
        ,interestRates.type #The type of interest rate (e.g. stable, fixed, variable, etc)
        ,interestRates.market
        ,interestRates.marketAsset
        ,interestRates.token
    ])

    #interestRateslist.sort_values(by=['interestRates_timestamp'])

    interestRatesDb=pd.concat([interestRateslist, interestRatesDb])
    interestRatesDbLength = len(interestRatesDb)
    interestRateListLength = len(interestRateslist)
    print("iterations: ", interestRateTicker, "Lines skipped: ",skipValue, "records collected: ", interestRateListLength)
    #print("iterations: ", interestRateTicker, "records collected: ", interestRateListLength, " - userdb length: ", interestRatesDbLength)
    interestRateTicker = interestRateTicker+1
    if interestRateListLength<1000: exit =True

print("Done. Total Records collected: ", interestRatesDbLength)




iterations:  0 Lines skipped:  0 records collected:  6
Done. Total Records collected:  6


In [90]:
interestRatesDb.head(10)

,interestRates_id,interestRates_rate,interestRates_duration,interestRates_maturityBlock,interestRates_side,interestRates_type,interestRates_market_id,interestRates_market_protocol_id,interestRates_market_name,interestRates_market_isActive,...,interestRates_token_symbol,interestRates_token_decimals,interestRates_token_type,interestRates_token_lastPriceUSD,interestRates_token_lastPriceBlockNumber,interestRates_token_market_id,interestRates_token_asset_id,interestRates_token_derivativeConversion,interestRates_token_totalSupply,interestRates_token_activeOracle_id
0,LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d38...,2.148783,None,None,LENDER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,XAI,18,Asset,0.997695,16595930,None,None,1.0,0,0xe37b8c83138caf12e57632d19c06eb561d47e423-0xd...
1,LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d38...,0.000000,None,None,LENDER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,WETH,18,Asset,1538.963318,16595930,None,None,1.0,0,None
2,LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d38...,0.000000,None,None,LENDER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,gOHM,18,Asset,2643.081034,16595930,None,None,1.0,0,0xba53d3a08fb8dd4309881b4e8a3bc386ea0ddb1f-0x0...
3,BORROWER-VARIABLE-0xc413dd03555f3eb29d834b482d...,3.277791,None,None,BORROWER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,XAI,18,Asset,0.997695,16595930,None,None,1.0,0,0xe37b8c83138caf12e57632d19c06eb561d47e423-0xd...
4,BORROWER-VARIABLE-0xc413dd03555f3eb29d834b482d...,0.000000,None,None,BORROWER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,WETH,18,Asset,1538.963318,16595930,None,None,1.0,0,None
5,BORROWER-VARIABLE-0xc413dd03555f3eb29d834b482d...,0.000000,None,None,BORROWER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,gOHM,18,Asset,2643.081034,16595930,None,None,1.0,0,0xba53d3a08fb8dd4309881b4e8a3bc386ea0ddb1f-0x0...


In [91]:
path =file+'/siloFinance_interestRates_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records.csv'
interestRatesDb.to_csv(path, index = False)

path =file+'/siloFinance_interestRates_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records_piped.csv'
interestRatesDb.to_csv(path, index = False, sep='|')
print('file saved')

********************************************************************
********************************************************************


LET'S LOOK AT VESTA WHILE WE'RE ON A RUN

Annoying: the schema for repay events is completely different from borrow and deposit. Can't union them into a single table without crunching it first. Argh.

Interest Rate data is weird, too. But I may not need it?

In [22]:
#https://api.thegraph.com/subgraphs/name/corerouter/vesta-finance
vesta = sg.load_api('https://api.thegraph.com/subgraphs/name/corerouter/vesta-finance')

In [23]:
##this captures the list of people who vestaBorrowed XAI against gOHM
vestaBorrowTicker = 0
vestaBorrowslist = pd.DataFrame()
vestaBorrowsDb = pd.DataFrame()
vestaBorrowListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaBorrowTicker)*(1000)
    vestaBorrows = vesta.Query.borrows(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1' } #gOHM market

    )

    vestaBorrowslist = sg.query_df([
        vestaBorrows.id
        ,vestaBorrows.hash
        ,vestaBorrows.logIndex
        ,vestaBorrows.blockNumber
        ,vestaBorrows.timestamp
        ,vestaBorrows.account
        ,vestaBorrows.market
        ,vestaBorrows.market.rates
        ,vestaBorrows.asset
        ,vestaBorrows.amount
        ,vestaBorrows.amountUSD
        ,vestaBorrows.position
    ])

    #vestaBorrowslist.sort_values(by=['vestaBorrows_timestamp'])

    vestaBorrowsDb=pd.concat([vestaBorrowsDb, vestaBorrowslist])
    recordTimestamp1 = vestaBorrowsDb.iat[vestaBorrowTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaBorrowsDbLength = len(vestaBorrowsDb)
    vestaBorrowListLength = len(vestaBorrowslist)
    recordID = vestaBorrowsDb.iat[vestaBorrowTicker,0]
    vestaBorrow = vestaBorrowsDb.iat[vestaBorrowTicker,1]
    print("iterations: ", vestaBorrowTicker, "Lines skipped: ",skipValue, "records collected: ", vestaBorrowListLength, " - latest recordID: ",vestaBorrow, " ", recordID , " - vestaBorrows DB length: ", vestaBorrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaBorrowTicker, "records collected: ", vestaBorrowListLength, " - userdb length: ", vestaBorrowsDbLength)
    vestaBorrowTicker = vestaBorrowTicker+1

    if daysAgo>daysLimit: exit =True
    if vestaBorrowListLength<1000: exit =True

#vestaBorrowsDbTimed.columns = vestaBorrowsDbTimed.columns.str.replace("borrows", "transaction")
vestaBorrowsDb['event']='borrow'
print("Done. Total Records collected: ", vestaBorrowsDbLength)

iterations:  0 Lines skipped:  0 records collected:  1000  - latest recordID:  0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e2fb6ee6429aaa36ded8ce3ce15d   borrow-0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e2fb6ee6429aaa36ded8ce3ce15d-38  - vestaBorrows DB length:  1000  - latest record from:  2023-02-09 12:33:03 1 days ago
iterations:  1 Lines skipped:  1000 records collected:  397  - latest recordID:  0x5d3025a16815324cf1afca689903a14d2621c19c95729e618fa373df3b736eee   borrow-0x5d3025a16815324cf1afca689903a14d2621c19c95729e618fa373df3b736eee-37  - vestaBorrows DB length:  1397  - latest record from:  2023-02-08 20:04:50 1 days ago
Done. Total Records collected:  1397


In [24]:
vestaBorrowsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(borrows_timestamp::int) dateTime "
               "From vestaBorrowsDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()

vestaBorrowsDbTimed.head(100)

,borrows_id,borrows_hash,borrows_logIndex,borrows_blockNumber,borrows_timestamp,borrows_account_id,borrows_account_positionCount,borrows_account_openPositionCount,borrows_account_closedPositionCount,borrows_account_depositCount,...,borrows_position_side,borrows_position_isCollateral,borrows_position_balance,borrows_position_depositCount,borrows_position_withdrawCount,borrows_position_borrowCount,borrows_position_repayCount,borrows_position_liquidationCount,event,dateTime
0,borrow-0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e...,0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e2fb6ee6...,38,59495981,1675974783,0x15bae977db3b62ceb5a67921fae589fd80624e8d,3,2,1,4,...,BORROWER,NaN,550000374479268793127538,0,0,4,150,0,borrow,2023-02-09 20:33:03
1,borrow-0x5d3025a16815324cf1afca689903a14d2621c...,0x5d3025a16815324cf1afca689903a14d2621c19c9572...,37,59294798,1675915490,0x7723c45fb37092ee65202db7a2fe562c5fe0ab48,9,4,5,5,...,BORROWER,NaN,600000064003338503382,0,0,3,0,0,borrow,2023-02-09 04:04:50
2,borrow-0x9285593b5e82d62c3211c6ea2b1ae37878091...,0x9285593b5e82d62c3211c6ea2b1ae378780919897063...,4,59127942,1675866088,0x515755b2c5a209976cf0de869c30f45ac7495a60,6,0,6,12,...,BORROWER,NaN,0,0,0,1,1,0,borrow,2023-02-08 14:21:28
3,borrow-0x6150d3df67aa2f3cb0c5041d516a4e0f3cffc...,0x6150d3df67aa2f3cb0c5041d516a4e0f3cffccfe70d4...,36,58934904,1675812656,0x394b67c6bc05abb14c73a57706dcd5cb85231c4e,2,2,0,1,...,BORROWER,NaN,1669620110651309523172,0,0,3,1,0,borrow,2023-02-07 23:30:56
4,borrow-0x66b8e892882e3f5d0a6bd0aa81ba9d5cec32e...,0x66b8e892882e3f5d0a6bd0aa81ba9d5cec32ec4cd786...,26,58785787,1675771382,0xe2a4de267cdd4ff5ed9ba13552f5c624b12db9b2,20,6,14,65,...,BORROWER,NaN,78903329999999999121255,0,0,6,1,0,borrow,2023-02-07 12:03:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,borrow-0x39e62f0bf2b26fb415c18eed751f3c6148f27...,0x39e62f0bf2b26fb415c18eed751f3c6148f271f70ca5...,23,47668312,1671580995,0xe2a4de267cdd4ff5ed9ba13552f5c624b12db9b2,20,6,14,65,...,BORROWER,NaN,78903329999999999121255,0,0,6,1,0,borrow,2022-12-21 00:03:15
96,borrow-0x965180bee5e4d28e5299cc0c996778fa816f3...,0x965180bee5e4d28e5299cc0c996778fa816f3b9646ec...,34,47474556,1671516834,0x2682efcc31de7a06ed3bf8b0be156ff88e1663a1,7,2,5,5,...,BORROWER,NaN,105000101450474620590108,0,0,11,0,0,borrow,2022-12-20 06:13:54
97,borrow-0xcd8fdb79f9c0f0ebda33b2ceb25fbc541a246...,0xcd8fdb79f9c0f0ebda33b2ceb25fbc541a24624636e3...,34,47370369,1671471134,0x53430bde5e57ed878406a20ca101b0853f173acf,4,2,2,2,...,BORROWER,NaN,3614750313057953658431,0,0,4,2,0,borrow,2022-12-19 17:32:14
98,borrow-0xb99ce9341425e4d0c4b48669847eebbe6a4f1...,0xb99ce9341425e4d0c4b48669847eebbe6a4f1cbfad75...,34,47344538,1671462599,0xf352e5320291298be60d00a015b27d3960f879fa,5,1,4,4,...,BORROWER,NaN,0,0,0,1,1,0,borrow,2022-12-19 15:09:59


In [25]:
path =file+'/vesta_borrow_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records.csv'
vestaBorrowsDbTimed.to_csv(path, index = False)

path =file+'/vesta_borrow_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records_piped.csv'
vestaBorrowsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [34]:
#this tracks the events for vestaDepositing gOHM for lending 
vestaDepositTicker = 0
vestaDepositslist = pd.DataFrame()
vestaDepositsDb = pd.DataFrame()
vestaDepositListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaDepositTicker)*(1000)
    vestaDeposits = vesta.Query.deposits(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1' } #gOHM market

    )

    vestaDepositslist = sg.query_df([
        vestaDeposits.id
        ,vestaDeposits.hash
        ,vestaDeposits.logIndex
        ,vestaDeposits.blockNumber
        ,vestaDeposits.timestamp
        ,vestaDeposits.account
        ,vestaDeposits.market
        ,vestaDeposits.market.rates
        ,vestaDeposits.asset
        ,vestaDeposits.amount
        ,vestaDeposits.amountUSD
        ,vestaDeposits.position
    ])

    #vestaDepositslist.sort_values(by=['vestaDeposits_timestamp'])

    vestaDepositsDb=pd.concat([vestaDepositsDb, vestaDepositslist])
    recordTimestamp1 = vestaDepositsDb.iat[vestaDepositTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaDepositsDbLength = len(vestaDepositsDb)
    vestaDepositListLength = len(vestaDepositslist)
    recordID = vestaDepositsDb.iat[vestaDepositTicker,0]
    vestaDeposit = vestaDepositsDb.iat[vestaDepositTicker,1]
    print("iterations: ", vestaDepositTicker, "Lines skipped: ",skipValue, "records collected: ", vestaDepositListLength, " - latest recordID: ",vestaDeposit, " ", recordID , " - vestaDeposits DB length: ", vestaDepositsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaDepositTicker, "records collected: ", vestaDepositListLength, " - userdb length: ", vestaDepositsDbLength)
    vestaDepositTicker = vestaDepositTicker+1
    if daysAgo>daysLimit: exit =True
    if vestaDepositListLength<1000: exit =True

#vestaDepositsDb.sort_values(by=['vestaDeposits_timestamp'])

vestaDepositsDb.columns = vestaDepositsDb.columns.str.replace("deposits", "transaction")
vestaDepositsDb['event']='deposit'
print("Done. Total Records collected: ", vestaDepositsDbLength)




iterations:  0 Lines skipped:  0 records collected:  1000  - latest recordID:  0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e2fb6ee6429aaa36ded8ce3ce15d   deposit-0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e2fb6ee6429aaa36ded8ce3ce15d-38  - vestaDeposits DB length:  1000  - latest record from:  2023-02-09 12:33:03 1 days ago
iterations:  1 Lines skipped:  1000 records collected:  333  - latest recordID:  0x5d3025a16815324cf1afca689903a14d2621c19c95729e618fa373df3b736eee   deposit-0x5d3025a16815324cf1afca689903a14d2621c19c95729e618fa373df3b736eee-37  - vestaDeposits DB length:  1333  - latest record from:  2023-02-08 20:04:50 1 days ago
Done. Total Records collected:  1333


In [35]:
vestaDepositsDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,deposit-0xa70b833620fa9e60985ec9d9aa6b9c6af9f5...,0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e2fb6ee6...,38,59495981,1675974783,0x15bae977db3b62ceb5a67921fae589fd80624e8d,3,2,1,4,...,NaN,LENDER,True,419221347099961371843,1,0,0,0,0,deposit
1,deposit-0x5d3025a16815324cf1afca689903a14d2621...,0x5d3025a16815324cf1afca689903a14d2621c19c9572...,37,59294798,1675915490,0x7723c45fb37092ee65202db7a2fe562c5fe0ab48,9,4,5,5,...,NaN,LENDER,True,395431000000000000,1,0,0,0,0,deposit
2,deposit-0xabfb1cba9108afe5c2dde3f21f5b4de92fae...,0xabfb1cba9108afe5c2dde3f21f5b4de92fae79791af2...,4,58531581,1675694548,0x1d883459bbc71042a9072f106427c4993de44794,2,2,0,2,...,NaN,LENDER,True,18790084000000000515,2,0,0,0,0,deposit
3,deposit-0xdb674a81e3a0fd547a3a2ace9befd38a3b86...,0xdb674a81e3a0fd547a3a2ace9befd38a3b867df74fc0...,30,58285118,1675618839,0x0fd84d7cb911728737556684050782b298f70f0f,7,2,5,5,...,NaN,LENDER,True,139919099999999990809,1,0,0,0,0,deposit
4,deposit-0xe1f1f1cfd372bbe64a06346c1dde1102e155...,0xe1f1f1cfd372bbe64a06346c1dde1102e155be4ec282...,34,58200953,1675595730,0x2682efcc31de7a06ed3bf8b0be156ff88e1663a1,7,2,5,5,...,NaN,LENDER,True,99538985999999994122,2,0,0,0,0,deposit
5,deposit-0x81d0e5f18de8d3995c911b0b022d2b051866...,0x81d0e5f18de8d3995c911b0b022d2b05186668fe6c0b...,23,58147623,1675579717,0x53430bde5e57ed878406a20ca101b0853f173acf,4,2,2,2,...,NaN,LENDER,True,2648200000000000000,1,0,0,0,0,deposit
6,deposit-0x16bfcc7fe7939a6c9935e5062c8067a57acd...,0x16bfcc7fe7939a6c9935e5062c8067a57acd4b32abb1...,42,58118599,1675569579,0xc6f84e7142eeaf930c62584f45ba7106bc3e2d68,2,2,0,1,...,NaN,LENDER,True,314789000000000000,1,0,0,0,0,deposit
7,deposit-0x61a65a717a84989f1315c17975c28846056a...,0x61a65a717a84989f1315c17975c28846056a95bb61c2...,42,58014267,1675533398,0xf5d0318692f822680b733955828a25507f031c95,2,0,2,1,...,1.675979e+09,LENDER,False,0,1,1,0,0,0,deposit
8,deposit-0xd076494b2508137add66e1bc117f7e6a2e7d...,0xd076494b2508137add66e1bc117f7e6a2e7da260823b...,34,57572941,1675390845,0xa80191fca50be00f8952c69232c93d57eeacaf6f,4,2,2,9,...,NaN,LENDER,True,837236991407343407276,1,0,0,0,0,deposit
9,deposit-0x553d1f6e555d93416a509488fd698a2651bb...,0x553d1f6e555d93416a509488fd698a2651bb5c39caa9...,42,57537289,1675375788,0xfa4dfbe7b012b1d92fdd5544f30518bb58bd41ce,2,0,2,1,...,1.675723e+09,LENDER,False,0,1,1,0,0,0,deposit


In [37]:
vestaDepositsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(transaction_timestamp::int) dateTime "
               "From vestaDepositsDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()

vestaDepositsDbTimed.head(100)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event,dateTime
0,deposit-0xa70b833620fa9e60985ec9d9aa6b9c6af9f5...,0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e2fb6ee6...,38,59495981,1675974783,0x15bae977db3b62ceb5a67921fae589fd80624e8d,3,2,1,4,...,LENDER,True,419221347099961371843,1,0,0,0,0,deposit,2023-02-09 20:33:03
1,deposit-0x5d3025a16815324cf1afca689903a14d2621...,0x5d3025a16815324cf1afca689903a14d2621c19c9572...,37,59294798,1675915490,0x7723c45fb37092ee65202db7a2fe562c5fe0ab48,9,4,5,5,...,LENDER,True,395431000000000000,1,0,0,0,0,deposit,2023-02-09 04:04:50
2,deposit-0xabfb1cba9108afe5c2dde3f21f5b4de92fae...,0xabfb1cba9108afe5c2dde3f21f5b4de92fae79791af2...,4,58531581,1675694548,0x1d883459bbc71042a9072f106427c4993de44794,2,2,0,2,...,LENDER,True,18790084000000000515,2,0,0,0,0,deposit,2023-02-06 14:42:28
3,deposit-0xdb674a81e3a0fd547a3a2ace9befd38a3b86...,0xdb674a81e3a0fd547a3a2ace9befd38a3b867df74fc0...,30,58285118,1675618839,0x0fd84d7cb911728737556684050782b298f70f0f,7,2,5,5,...,LENDER,True,139919099999999990809,1,0,0,0,0,deposit,2023-02-05 17:40:39
4,deposit-0xe1f1f1cfd372bbe64a06346c1dde1102e155...,0xe1f1f1cfd372bbe64a06346c1dde1102e155be4ec282...,34,58200953,1675595730,0x2682efcc31de7a06ed3bf8b0be156ff88e1663a1,7,2,5,5,...,LENDER,True,99538985999999994122,2,0,0,0,0,deposit,2023-02-05 11:15:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,deposit-0x92fb45376ce2474a9812168f9be4d401a44c...,0x92fb45376ce2474a9812168f9be4d401a44cb0bd62d9...,23,43909041,1670273939,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,152,0,152,132,...,LENDER,False,0,2,2,0,0,0,deposit,2022-12-05 20:58:59
96,deposit-0xf4eb0268999a30bf9a33fc801dfbe2dc65c2...,0xf4eb0268999a30bf9a33fc801dfbe2dc65c2e18ef07f...,4,43845816,1670255076,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,152,0,152,132,...,LENDER,False,0,1,1,0,0,0,deposit,2022-12-05 15:44:36
97,deposit-0xd0d6884309c0aae435c50d7884795893b57e...,0xd0d6884309c0aae435c50d7884795893b57e45b10d49...,4,43810557,1670245899,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,152,0,152,132,...,LENDER,False,0,1,2,0,0,0,deposit,2022-12-05 13:11:39
98,deposit-0x2fca7637117ca01aed11a872a21628aa6ab5...,0x2fca7637117ca01aed11a872a21628aa6ab5d13e327a...,4,43667038,1670195903,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,152,0,152,132,...,LENDER,False,0,1,1,0,0,0,deposit,2022-12-04 23:18:23


In [38]:
path =file+'/vesta_deposit_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records.csv'
vestaBorrowsDbTimed.to_csv(path, index = False)

path =file+'/vesta_deposit_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records_piped.csv'
vestaBorrowsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [45]:
##this tracks the ENTIRE list of events for vestaRepaying the VST vestaRepayed against gOHM
vestaRepayTicker = 0
vestaRepayslist = pd.DataFrame()
vestaRepaysDb = pd.DataFrame()
vestaRepayListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaRepayTicker)*(1000)
    vestaRepays = silo.Query.repays(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    vestaRepayslist = sg.query_df([
        vestaRepays.id
        ,vestaRepays.hash
        ,vestaRepays.logIndex
        ,vestaRepays.blockNumber
        ,vestaRepays.timestamp
        ,vestaRepays.account
        ,vestaRepays.market
        ,vestaRepays.market.rates
        ,vestaRepays.asset
        ,vestaRepays.amount
        ,vestaRepays.amountUSD
        ,vestaRepays.position
    ])

    #vestaRepayslist.sort_values(by=['vestaRepays_timestamp'])

    vestaRepaysDb=pd.concat([vestaRepaysDb, vestaRepayslist])
    recordTimestamp1 = vestaRepaysDb.iat[vestaRepayTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaRepaysDbLength = len(vestaRepaysDb)
    vestaRepayListLength = len(vestaRepayslist)
    recordID = vestaRepaysDb.iat[vestaRepayTicker,0]
    vestaRepay = vestaRepaysDb.iat[vestaRepayTicker,1]
    print("iterations: ", vestaRepayTicker, "Lines skipped: ",skipValue, "records collected: ", vestaRepayListLength, " - latest recordID: ",vestaRepay, " ", recordID , " - vestaRepays DB length: ", vestaRepaysDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaRepayTicker, "records collected: ", vestaRepayListLength, " - userdb length: ", vestaRepaysDbLength)
    vestaRepayTicker = vestaRepayTicker+1

    if daysAgo>daysLimit: exit =True
    if vestaRepayListLength<1000: exit =True

vestaRepaysDb.columns = vestaRepaysDb.columns.str.replace("repays", "transaction")
vestaRepaysDb['event']='Repay'
print("Done. Total Records collected: ", vestaRepaysDbLength)

iterations:  0 Lines skipped:  0 records collected:  12  - latest recordID:  0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad01a68715d0385e8b26cd   0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad01a68715d0385e8b26cd-242  - vestaRepays DB length:  12  - latest record from:  2022-11-25 13:03:47 77 days ago
Done. Total Records collected:  12


In [46]:
vestaRepaysDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,Repay
1,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,Repay
2,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,Repay
3,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,Repay
4,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,Repay
5,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,Repay
6,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,1669248167,BORROWER,True,0,0,0,1,1,0,Repay
7,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,1669248167,BORROWER,True,0,0,0,1,1,0,Repay
8,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,1669248167,BORROWER,True,0,0,0,1,1,0,Repay
9,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,1669248167,BORROWER,True,0,0,0,1,1,0,Repay


In [47]:
vestaRepaysDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(transaction_timestamp::int) dateTime "
               "From vestaRepaysDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()

vestaRepaysDbTimed.head(100)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event,dateTime
0,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,BORROWER,True,0,0,0,1,1,0,Repay,2022-11-25 21:03:47
1,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,BORROWER,True,0,0,0,1,1,0,Repay,2022-11-25 21:03:47
2,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,BORROWER,True,0,0,0,1,1,0,Repay,2022-11-25 21:03:47
3,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,BORROWER,True,0,0,0,1,1,0,Repay,2022-11-25 21:03:47
4,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,BORROWER,True,0,0,0,1,1,0,Repay,2022-11-25 21:03:47
5,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,BORROWER,True,0,0,0,1,1,0,Repay,2022-11-25 21:03:47
6,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,BORROWER,True,0,0,0,1,1,0,Repay,2022-11-24 00:02:47
7,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,BORROWER,True,0,0,0,1,1,0,Repay,2022-11-24 00:02:47
8,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,BORROWER,True,0,0,0,1,1,0,Repay,2022-11-24 00:02:47
9,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,BORROWER,True,0,0,0,1,1,0,Repay,2022-11-24 00:02:47


In [ ]:
path =file+'/vesta_repay_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records.csv'
vestaBorrowsDbTimed.to_csv(path, index = False)

path =file+'/vesta_repay_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records_piped.csv'
vestaBorrowsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

In [49]:
vestaBorrowDepositEventsDb = pd.DataFrame()
vestaBorrowDepositEventsDb=pd.concat([vestaDepositsDbTimed, vestaBorrowsDbTimed])#,vestaRepaysDbTimed])
vestaBorrowDepositEventsDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,borrows_position_blockNumberClosed,borrows_position_timestampClosed,borrows_position_side,borrows_position_isCollateral,borrows_position_balance,borrows_position_depositCount,borrows_position_withdrawCount,borrows_position_borrowCount,borrows_position_repayCount,borrows_position_liquidationCount
0,deposit-0xa70b833620fa9e60985ec9d9aa6b9c6af9f5...,0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e2fb6ee6...,38.0,59495981.0,1.675975e+09,0x15bae977db3b62ceb5a67921fae589fd80624e8d,3.0,2.0,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,deposit-0x5d3025a16815324cf1afca689903a14d2621...,0x5d3025a16815324cf1afca689903a14d2621c19c9572...,37.0,59294798.0,1.675915e+09,0x7723c45fb37092ee65202db7a2fe562c5fe0ab48,9.0,4.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,deposit-0xabfb1cba9108afe5c2dde3f21f5b4de92fae...,0xabfb1cba9108afe5c2dde3f21f5b4de92fae79791af2...,4.0,58531581.0,1.675695e+09,0x1d883459bbc71042a9072f106427c4993de44794,2.0,2.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,deposit-0xdb674a81e3a0fd547a3a2ace9befd38a3b86...,0xdb674a81e3a0fd547a3a2ace9befd38a3b867df74fc0...,30.0,58285118.0,1.675619e+09,0x0fd84d7cb911728737556684050782b298f70f0f,7.0,2.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,deposit-0xe1f1f1cfd372bbe64a06346c1dde1102e155...,0xe1f1f1cfd372bbe64a06346c1dde1102e155be4ec282...,34.0,58200953.0,1.675596e+09,0x2682efcc31de7a06ed3bf8b0be156ff88e1663a1,7.0,2.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,deposit-0x81d0e5f18de8d3995c911b0b022d2b051866...,0x81d0e5f18de8d3995c911b0b022d2b05186668fe6c0b...,23.0,58147623.0,1.675580e+09,0x53430bde5e57ed878406a20ca101b0853f173acf,4.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,deposit-0x16bfcc7fe7939a6c9935e5062c8067a57acd...,0x16bfcc7fe7939a6c9935e5062c8067a57acd4b32abb1...,42.0,58118599.0,1.675570e+09,0xc6f84e7142eeaf930c62584f45ba7106bc3e2d68,2.0,2.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,deposit-0x61a65a717a84989f1315c17975c28846056a...,0x61a65a717a84989f1315c17975c28846056a95bb61c2...,42.0,58014267.0,1.675533e+09,0xf5d0318692f822680b733955828a25507f031c95,2.0,0.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,deposit-0xd076494b2508137add66e1bc117f7e6a2e7d...,0xd076494b2508137add66e1bc117f7e6a2e7da260823b...,34.0,57572941.0,1.675391e+09,0xa80191fca50be00f8952c69232c93d57eeacaf6f,4.0,2.0,2.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,deposit-0x553d1f6e555d93416a509488fd698a2651bb...,0x553d1f6e555d93416a509488fd698a2651bb5c39caa9...,42.0,57537289.0,1.675376e+09,0xfa4dfbe7b012b1d92fdd5544f30518bb58bd41ce,2.0,0.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
path =file+'/vesta_BorrowDeposit_Events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records.csv'
vestaBorrowDepositEventsDb.to_csv(path, index = False)

path =file+'/vesta_repay_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records_piped.csv'
vestaBorrowDepositEventsDb.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [53]:
#let's pull interest rate data
interestRateTicker = 0
interestRateslist = pd.DataFrame()
interestRatesDb = pd.DataFrame()
interestRateListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (interestRateTicker)*(1000)
    interestRates = vesta.Query.interestRates(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        #where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    interestRateslist = sg.query_df([
        interestRates.id
        ,interestRates.rate #5.21% should be stored as 5.21
        ,interestRates.duration #in days
        ,interestRates.maturityBlock #Maturity of the loan in block height. Only applies to fixed term lending (e.g. Notional)
        ,interestRates.side #The party the interest is paid to / received from
        ,interestRates.type #The type of interest rate (e.g. stable, fixed, variable, etc)
    ])

    #interestRateslist.sort_values(by=['interestRates_timestamp'])

    interestRatesDb=pd.concat([interestRateslist, interestRatesDb])
    interestRatesDbLength = len(interestRatesDb)
    interestRateListLength = len(interestRateslist)
    print("iterations: ", interestRateTicker, "Lines skipped: ",skipValue, "records collected: ", interestRateListLength)
    #print("iterations: ", interestRateTicker, "records collected: ", interestRateListLength, " - userdb length: ", interestRatesDbLength)
    interestRateTicker = interestRateTicker+1
    if interestRateListLength<1000: exit =True

print("Done. Total Records collected: ", interestRatesDbLength)




iterations:  0 Lines skipped:  0 records collected:  1000
iterations:  1 Lines skipped:  1000 records collected:  575
Done. Total Records collected:  1575


In [54]:
interestRatesDb.head(5)

,interestRates_id,interestRates_rate,interestRates_duration,interestRates_maturityBlock,interestRates_side,interestRates_type
0,BORROWER-STABLE-0x6c2c06790b3e3e3c38e12ee22f81...,0.0,None,None,BORROWER,STABLE
1,BORROWER-STABLE-0x6c2c06790b3e3e3c38e12ee22f81...,0.0,None,None,BORROWER,STABLE
2,BORROWER-STABLE-0x6c2c06790b3e3e3c38e12ee22f81...,0.0,None,None,BORROWER,STABLE
3,BORROWER-STABLE-0x6c2c06790b3e3e3c38e12ee22f81...,0.0,None,None,BORROWER,STABLE
4,BORROWER-STABLE-0x6c2c06790b3e3e3c38e12ee22f81...,0.0,None,None,BORROWER,STABLE


##this WILL Capture the list of people who lended gOHM for others to borrow
lendTicker = 0
lendslist = pd.DataFrame()
lendsDb = pd.DataFrame()
lendListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (lendTicker)*(1000)
    lends = silo.Query.lends(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    lendslist = sg.query_df([
        lends.id
        ,lends.hash
        ,lends.logIndex
        ,lends.blockNumber
        ,lends.timestamp
        ,lends.account
        ,lends.market
        ,lends.asset
        ,lends.amount
        ,lends.amountUSD
    ])

    #lendslist.sort_values(by=['lends_timestamp'])

    lendsDb=pd.concat([lendsDb, lendslist])
    recordTimestamp1 = lendsDb.iat[lendTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    lendsDbLength = len(lendsDb)
    lendListLength = len(lendslist)
    recordID = lendsDb.iat[lendTicker,0]
    lend = lendsDb.iat[lendTicker,1]
    print("iterations: ", lendTicker, "Lines skipped: ",skipValue, "records collected: ", lendListLength, " - latest recordID: ",lend, " ", recordID , " - lends DB length: ", lendsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", lendTicker, "records collected: ", lendListLength, " - userdb length: ", lendsDbLength)
    lendTicker = lendTicker+1
    if daysAgo>daysLimit: exit =True
    if lendListLength<1000 exit =True

#lendsDb.sort_values(by=['lends_timestamp'])
print("Done. Total Records collected: ", lendsDbLength)